In [1]:
from Controller import TraceClassifierController
from Utils.SaveUtils import load_parameters
from Parameters import TrainingParameters
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
# %tensorboard --logdir logs/gradient_tape --host localhost --port 8088

In [3]:
parameters = TrainingParameters()
tf.random.set_seed(parameters.dataset_split_seed)
np.random.seed(parameters.dataset_split_seed)
controller = TraceClassifierController(parameters = parameters)


| Running on /job:localhost/replica:0/task:0/device:CPU:0  
parsing log, completed traces :: 100%|██████████| 13087/13087 [00:06<00:00, 2168.14it/s]

| Preprocessed data saved successfully 


In [4]:
controller.show_model_info()

Model: "baseline_lstm_with_resource_valid_trace_cf"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1664      
_________________________________________________________________
embedding_1 (Embedding)      multiple                  9216      
_________________________________________________________________
lstm (LSTM)                  multiple                  328704    
_________________________________________________________________
lstm_1 (LSTM)                multiple                  525312    
_________________________________________________________________
lstm_2 (LSTM)                multiple                  394240    
_________________________________________________________________
lstm_3 (LSTM)                multiple                  525312    
_________________________________________________________________
sequential (Sequential) 

In [5]:
controller.train()


| Training records in logs/gradient_tape/20210625-165638 

| Total epochs: 5 

| Total steps: 1230 

| Start epoch 0 

| Start epoch 1 

| Start epoch 2 

| Evaluation result | Loss [0.3683] | Accuracy [0.6493]  

| Start epoch 3 

| Start epoch 4 

| Evaluation result | Loss [0.0322] | Accuracy [0.6493]  


KeyboardInterrupt: 

In [7]:
## Accuracy in first round 0.7595908

In [ ]:
###### Save modelz
controller.save_training_result("classify_valid_trace.ipynb")

In [10]:
# trainer.load_trained_model("./SavedModels/0.7700_Diabetes_BaseNNModel_2021-05-30 04:41:24.179089")

In [17]:
controller.y_true[5]

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1], dtype=int32)

In [18]:
controller.y_pred[5]

array([[0.7195055 ],
       [0.99133193],
       [0.99977666],
       [0.999992  ],
       [0.99997723],
       [0.9999832 ],
       [0.9999937 ],
       [0.9999969 ],
       [0.99999785],
       [0.999995  ],
       [0.9999987 ],
       [0.9999995 ],
       [0.9999988 ],
       [0.9999987 ],
       [0.9999999 ],
       [0.9999989 ],
       [0.9999995 ],
       [0.99999976],
       [0.99999976],
       [0.9999993 ],
       [0.99999857],
       [0.99999964],
       [0.9999999 ],
       [0.9999999 ],
       [0.9999999 ],
       [1.        ],
       [0.99999976],
       [0.9999999 ],
       [0.9999999 ],
       [1.        ],
       [1.        ],
       [0.9999999 ],
       [0.9999999 ],
       [0.9999999 ],
       [0.9999999 ],
       [1.        ],
       [1.        ],
       [0.9999999 ],
       [1.        ],
       [1.        ],
       [0.9999999 ],
       [1.        ],
       [0.9999987 ],
       [1.        ],
       [0.9999999 ],
       [0.9999999 ],
       [0.99999964],
       [1.   